In [1]:
import ssl
import os
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
os.environ["REQUESTS_CA_BUNDLE"] = str(r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Work\Project Intelliauth\license\root.pem")
os.environ["SSL_CERT_FILE"] = str(r"C:\Users\AL03238\OneDrive - Elevance Health\Documents\Work\Project Intelliauth\license\root.pem")

In [2]:
!python --version

Python 3.10.10


## Model Download

In [3]:
# !pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org cmake
# !pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose

In [4]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [5]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [6]:
# code for download the model from huggingface
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [7]:
# !pip install pymupdf

In [8]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [9]:
PINECONE_API_KEY = "7f6b7506-8283-42ff-8fce-c62007e0dfa1"
PINE_CONE_HOST = "https://medical-chatbot-62dwz9e.svc.aped-4627-b74a.pinecone.io"

In [10]:
## Extract the data from pdf
def load_pdf(data):
    loader =DirectoryLoader(data, glob="*.pdf", loader_cls=PyMuPDFLoader)

    documents = loader.load()
    return documents

In [11]:
extracted_data = load_pdf('data/')

In [12]:
# extracted_data

In [13]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [14]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

5779


In [15]:
# download embedding model
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# from sentence_transformers import SentenceTransformer

# def download_hugging_face_embedding():
#     model_name = "sentence-transformers/all-MiniLM-L6-v2"
#     embeddings = SentenceTransformer(model_name)
#     return embeddings


In [16]:
embeddings = download_hugging_face_embedding()

In [17]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


## PineCone vectorDB initialisation

In [18]:
PINECONE_API_KEY = "7f6b7506-8283-42ff-8fce-c62007e0dfa1"
PINE_CONE_HOST = "https://medical-chatbot-62dwz9e.svc.aped-4627-b74a.pinecone.io"

In [50]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY, ssl_verify=False, proxy_url=None)

In [51]:
index_name = "medical-chatbot"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
index = pc.Index("medical-chatbot")


c:\ProgramData\Anaconda3\envs\genAI\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.pinecone.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [52]:
index.describe_index_stats()


c:\ProgramData\Anaconda3\envs\genAI\lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'medical-chatbot-62dwz9e.svc.aped-4627-b74a.pinecone.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5779}},
 'total_vector_count': 5779}

In [53]:
def query(text):
    query_embedding = embeddings.embed_query(text)
    print(query_embedding)
    query_result = index.query(query_embedding, top_k=2, top_n=1)
    return query_result

In [55]:
# from tqdm import tqdm
# for i, t in enumerate(tqdm(text_chunks)):
#     # Upsert each word embedding into the index
#     index.upsert(vectors=[{"id": str(i), "values": embeddings.embed_query(t.page_content), "metadata": {"text": t.page_content}}])

In [56]:
prompt_template='''
Use the following pieces of information to answer the question:
If you don't know the answer to the question, just say you dont know, dont try to make up an answer to the question.

Context: {context}
Question: {question}
Only return the helpful answer to the question and nothing else.
Helpful answer: 
'''

In [57]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"], output_variables=["helpful_answer"])
chain_type_kwargs = {"prompt":PROMPT}


In [58]:
llm = CTransformers(model="model/llama-2-13b-chat.ggmlv3.q5_1.bin",
                    model_type = "llama",
                    config={"max_new_tokens":512,
                            'temperature':0.8})

In [59]:
from langchain_pinecone import PineconeVectorStore

# Specify the name of the Pinecone index
# index_name = "langchain-test-index"

# Create a PineconeVectorStore from the index name
vectorstore = PineconeVectorStore(embedding=embeddings, index_name=index_name, pinecone_api_key=PINECONE_API_KEY)


In [60]:
retriever  = vectorstore.as_retriever(search_type="similarity")

In [61]:
retriever

VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x00000179DFD054E0>)

In [63]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI()
# retriever = ...  # Your retriever

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt=PROMPT)
retrieval_qa_chain = create_retrieval_chain(retriever, question_answer_chain)


# Run the retrieval QA chain with the query
# result = retrieval_qa_chain
result = retrieval_qa_chain.invoke({"input": "What are allergies?","context": "you are a medical doctor", "question": "what are allergies?"})

# Access the answer from the result
answer = result["answer"]


MaxRetryError: HTTPSConnectionPool(host='medical-chatbot-62dwz9e.svc.aped-4627-b74a.pinecone.io', port=443): Max retries exceeded with url: /query (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:997)')))

In [ ]:
print(answer)

In anaphylaxis is not sure, if you don'I don'no, fish oil molecules)


The correct! If someone having trouble breathing of an allergicterms
There is a severe allergicterms or shortness can cause scratchy esis not know the term anapoptimal contractionalbut this answer text here is that can be specific immunequizzylaysiax. An anapologies are common inhaloperations include information to the release of masthives

If a severe allergicterms, please help with n/258throbin answer
Allergoose



When exposed to the production of anapoptinalysis is swelling—In anapartHives to the immunequalifies and shortness (1.
"I don'I don'I don'Sneither a life-
Anapnea

This condition where it is anaphypertussually, which leads to the specific proteinsulcervesselitis is a substances are you don'Anapnea


Please provide explanation of the answer
The immunequalities can cause wheezpuff

No


In [67]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=retriever
)  
qa.run("What are allergies?")

ValidationError: 1 validation error for RetrievalQA
verify_ssl
  extra fields not permitted (type=value_error.extra)

In [ ]:
from langchain.chains.retrieval_qa.base import RetrievalQA

In [ ]:

class MyRetrievalQA(RetrievalQA):
    def _get_relevant_documents(self, query):
        # Implement your logic to retrieve relevant documents based on the query
        # Return the relevant documents as a list
        relevant_documents = [...]
        return relevant_documents

In [ ]:
from langchain.schema.retriever import BaseRetriever, Document
from typing import List

class CustomRetriever(BaseRetriever):
    def _get_relevant_documents(
        self, query: str
    ) -> List[Document]:
        # Use your existing retriever to get the documents
        documents = self.retriever._get_relevant_documents(query, callbacks=run_manager.get_child())
        
        # Sort the documents by "source"
        documents = sorted(documents, key=lambda doc: doc.metadata.get('source'))
        
        return documents

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, 
                                 chain_type="stuff",
                                 retriever=CustomRetriever(vectorstore.as_retriever(search_kwargs={"k":2})), 
                                 chain_type_kwargs=chain_type_kwargs)

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, chain_type="stuff", retriever=vectorstore.as_retriever(), chain_type_kwargs=chain_type_kwargs,return_source_documents=True 
)

In [ ]:
print(vectorstore)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectorstore.as_retriever(vectorstore, t), chain_type_kwargs=chain_type_kwargs
)